## 의미 기반 Chunking 소개

Text chunking은 RAG(Retrieval-Augmented Generation) 시스템에서 꼭 필요한 과정으로, 긴 텍스트를 더 의미 있는 단위로 나누어 검색 정확도를 높이는 데 사용됩니다.
고정된 길이로 나누는 방식과 달리, **의미 기반 chunking**은 문장 간의 **내용 유사도**를 기준으로 텍스트를 나눕니다.

### 분할 기준 방법:

* **백분위(Percentile)**: 모든 문장 간 유사도 차이 중 X번째 백분위 값을 구하고, 이보다 큰 폭으로 유사도가 떨어지는 지점을 기준으로 분할합니다.
* **표준편차(Standard Deviation)**: 전체 평균보다 X 표준편차만큼 유사도가 급격히 떨어지는 지점을 기준으로 분할합니다.
* **사분위 범위(IQR)**: Q3(3사분위) - Q1(1사분위)로 계산된 IQR 값을 기준으로, 이보다 큰 차이가 발생하는 지점을 분할 지점으로 사용합니다.

이 노트북에서는 \*\*백분위 방식(percentile method)\*\*을 활용한 의미 기반 chunking을 구현하고, 예시 텍스트를 이용해 성능을 평가합니다.


## 환경 설정하기
필요한 라이브러리를 가져오는 것부터 시작합니다.

In [5]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import json

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('api_key')

## PDF 파일에서 텍스트 추출하기

In [2]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## OpenAI API 클라이언트 설정하기

In [6]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## 임베딩 만들기
텍스트를 문장으로 분할하여 임베딩을 생성합니다.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

sentences = extracted_text.split(". ")
# 배치 크기를 줄여 임베딩을 생성합니다.
embeddings = create_embeddings(embedding_model, sentences, device=device, batch_size=4)

print(f"Generated {len(embeddings)} sentence embeddings.")

## 문장간 유사도 차이 계산하기
연속된 문장 사이의 코사인 유사도를 계산합니다.

In [14]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 연속된 문장 사이의 코사인 유사도를 계산합니다.
similarities = [cosine_similarity(embeddings[i], embeddings[i + 1]) for i in range(len(embeddings) - 1)]

## 시맨틱 청킹 구현하기
중단점을 찾는 세 가지 방법을 구현합니다.

In [15]:
def compute_breakpoints(similarities, method="percentile", threshold=90):
    """
    유사도가 감소하는 부분을 기준으로 청킹 지점을 계산합니다.

    Args:
    similarities (List[float]): 문장 간 유사도 점수 목록입니다.
    method (str): 'percentile', 'standard_deviation', 'interquartile'
    threshold (float): 임계값 

    Returns:
    List[int]: 청킹 지점의 인덱스 목록입니다.
    """
    # 선택된 방법에 따라 임계값을 계산합니다.
    if method == "percentile":
        # 유사도 점수의 X번째 백분위 값을 계산합니다.
        threshold_value = np.percentile(similarities, threshold)
    elif method == "standard_deviation":
        # 유사도 점수의 평균과 표준편차를 계산합니다.
        mean = np.mean(similarities)
        std_dev = np.std(similarities)
        # 임계값을 평균에서 X 표준편차만큼 뺍니다.
        threshold_value = mean - (threshold * std_dev)
    elif method == "interquartile":
        # 첫 번째와 세 번째 사분위수(Q1과 Q3)를 계산합니다.
        q1, q3 = np.percentile(similarities, [25, 75])
        # IQR 규칙을 사용하여 임계값을 계산합니다.
        threshold_value = q1 - 1.5 * (q3 - q1)
    else:
        # 유효하지 않은 방법이 제공되면 오류를 발생시킵니다.
        raise ValueError("Invalid method. Choose 'percentile', 'standard_deviation', or 'interquartile'.")

    # 유사도가 임계값 아래로 떨어지는 인덱스를 식별합니다.
    return [i for i, sim in enumerate(similarities) if sim < threshold_value]

# 백분위 방법을 사용하여 임계값 90에서 청킹 지점을 계산합니다.
breakpoints = compute_breakpoints(similarities, method="percentile", threshold=90)

## 텍스트를 시맨틱 청크로 분할하기
계산된 청크 기준으로 텍스트를 분할합니다.

In [ ]:
def split_into_chunks(sentences, breakpoints):
    """
    Splits sentences into semantic chunks.

    Args:
    sentences (List[str]): List of sentences.
    breakpoints (List[int]): Indices where chunking should occur.

    Returns:
    List[str]: List of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks
    start = 0  # Initialize the start index

    # Iterate through each breakpoint to create chunks
    for bp in breakpoints:
        # Append the chunk of sentences from start to the current breakpoint
        chunks.append(". ".join(sentences[start:bp + 1]) + ".")
        start = bp + 1  # Update the start index to the next sentence after the breakpoint

    # Append the remaining sentences as the last chunk
    chunks.append(". ".join(sentences[start:]))
    return chunks  # Return the list of chunks

# Create chunks using the split_into_chunks function
text_chunks = split_into_chunks(sentences, breakpoints)

# Print the number of chunks created
print(f"Number of semantic chunks: {len(text_chunks)}")

# Print the first chunk to verify the result
print("\nFirst text chunk:")
print(text_chunks[0])


## 시맨틱 청크에 대한 임베딩 생성하기
나중에 검색할 수 있도록 각 청크에 대한 임베딩을 생성합니다.

In [ ]:
# create_embedings 함수를 사용하여 청크 임베딩 만들기

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 모델 로드
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

print("문장 갯수:", len(text_chunks))

# 배치 크기를 줄여 임베딩을 생성합니다.
embeddings = create_embeddings(embedding_model, text_chunks, device=device, batch_size=4)

print(f"Generated {len(embeddings)} sentence embeddings.")

## 시맨틱 검색 수행하기
코사인 유사도를 구현하여 사용자 쿼리에 가장 관련성이 높은 텍스트 청크를 찾습니다.

In [18]:
def semantic_search(query, text_chunks, chunk_embeddings, k=5):
    """
    주어진 쿼리와 임베딩을 사용하여 텍스트 청크에서 시맨틱 검색을 수행합니다.

    Args:
    query (str): 시맨틱 검색을 위한 쿼리입니다.
    text_chunks (List[str]): 검색할 텍스트 청크 리스트입니다.
    embeddings (List[dict]): 텍스트 청크에 대한 임베딩 리스트입니다.
    k (int): 상위 k개의 관련 텍스트 청크를 반환합니다. 기본값은 5입니다.

    Returns:
    List[str]: 쿼리와 가장 유사한 상위 k개의 텍스트 청크 리스트입니다.
    """
    # 쿼리에 대한 임베딩을 생성합니다.
    query_embedding = create_embeddings(embedding_model, [query])[0]
    
    # 쿼리 임베딩과 각 청크 임베딩 간의 코사인 유사도를 계산합니다.
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    # 가장 유사한 상위 k개의 청크의 인덱스를 추출합니다.
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # 가장 유사한 상위 k개의 청크를 반환합니다.
    return [text_chunks[i] for i in top_indices]

In [ ]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

In [ ]:
# 평가 데이터에서 첫 번째 쿼리를 추출합니다.
query = df['query'][0]

# 연관 있는 2개의 chunk를 찾는다
top_chunks = semantic_search(query, text_chunks, embeddings, k=3)

# 쿼리 출력
print(f"Query: {query}")

# 쿼리와 연관있는 2개의 chunk 출력
for i, chunk in enumerate(top_chunks):
    print(f"Context {i+1}:\n{chunk}\n{'='*40}")

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(system_prompt, user_message ,model_name='gpt-4.1-mini'):
    
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},# Define the system prompt for the AI assistant
            {"role": "user", "content": user_message}
        ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 검색된 청크를 기반으로 사용자 프롬프트를 생성합니다.
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\n질문: {query}"

# AI 어시스턴트의 시스템 프롬프트를 정의합니다.
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt,'gpt-4.1-nano-2025-04-14')

In [ ]:
print(ai_response.choices[0].message.content)

## 생성 응답 평가하기
생서 응답을 예상 답변과 비교하여 점수를 부여합니다.

In [ ]:
# 평가 시스템의 시스템 프롬프트를 정의합니다.
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# 사용자 쿼리, AI 응답, 실제 응답, 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다.
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][0]}\n{evaluate_system_prompt}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답을 생성합니다.
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

# 평가 응답을 출력합니다.
print(evaluation_response.choices[0].message.content)

## 전체 추론 및 평가

In [ ]:
# 생성된 답변과 실제 답변을 비교하여 점수를 부여합니다.
result = []

for i in range(len(df)):
    query = df['query'][i]
    top_chunks = semantic_search(query, text_chunks, embeddings, k=3)

    # 검색된 청크를 기반으로 사용자 프롬프트를 생성합니다.
    user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
    user_prompt = f"{user_prompt}\n질문: {query}"

    # AI 어시스턴트의 시스템 프롬프트를 정의합니다.
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

    # response 생성
    ai_response = generate_response(system_prompt, user_prompt, 'gpt-4.1-mini')

    print(ai_response.choices[0].message.content)

    # 평가 프롬프트를 정의합니다.
    evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5. \n The answer should only output numbers and should not output any words."

    # 사용자 쿼리, AI response, 실제 응답, 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다.
    evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][i]}\n{evaluate_system_prompt}"

    # 평가 응답 생성
    evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

    # 평가 응답을 출력합니다.
    print(evaluation_response.choices[0].message.content)
    result.append(evaluation_response.choices[0].message.content)

In [42]:
import numpy as np
result = [float(x.replace('\n','').replace('Score: ',''))for x in result]


In [ ]:
np.average(result)